In [1]:
import numpy as np
import pandas as pd
import json

idf = pd.read_csv('../../data/tt_inscripciones_samp.csv')

In [2]:
idf.head().T

,0,1,2,3,4
usu_estado_civil,S,S,S,S,S
usu_fecha_nac,10/5/1998 0:00:00,09/04/2002 0:00,20/09/2002 0:00,12/06/1985,06/05/2002
ins_sexo,MUJER,HOMBRE,MUJER,MUJER,MUJER
ins_fecha,25/4/2019 18:30:37,15/01/2021 1:03,17/01/2021 1:35,07/12/2019,05/12/2019
pais_res,ECUADOR,ECUADOR,ECUADOR,ECUADOR,ECUADOR
cod_provincia_reside,10.0,9.0,4.0,9.0,9.0
cod_canton_reside,1006.0,922.0,401.0,901.0,919.0
cod_parroquia_reside,100650.0,92250.0,40102.0,90114.0,91953.0
ins_autoidentificacion,MESTIZO,Montubio/a,Mestizo/a,MULATO,MONTUBIO
cod_final,1416621074.0,2397810938.0,2548550429.0,2362460974.0,2428020938.0


#### Duplicates

In [3]:
# No apparent duplicates in this subset
def get_shape(df):
    print(df.shape)
    return df

(idf
 .pipe(get_shape)
 .drop_duplicates()
 .pipe(get_shape)
 .sample()
)

(539535, 14)
(539455, 14)


,usu_estado_civil,usu_fecha_nac,ins_sexo,ins_fecha,pais_res,cod_provincia_reside,cod_canton_reside,cod_parroquia_reside,ins_autoidentificacion,cod_final,periodo,escolar_no_escolar,internet_domicilio,computadora_domicilio
408900,S,3/4/2000,HOMBRE,1/3/2019,ECUADOR,13.0,1309.0,130902.0,MESTIZO,1.697051e+09,17,NaN,NaN,NaN


### Understanding our data

#### Missing Values

In [4]:
(idf
 .isna()
 .groupby(idf.periodo)
 .mean()
 .style
 .format(lambda fl: f'{fl * 100:.2f} %')
)

,usu_estado_civil,usu_fecha_nac,ins_sexo,ins_fecha,pais_res,cod_provincia_reside,cod_canton_reside,cod_parroquia_reside,ins_autoidentificacion,cod_final,periodo,escolar_no_escolar,internet_domicilio,computadora_domicilio
periodo,,,,,,,,,,,,,,
17,0.40 %,0.37 %,0.00 %,0.00 %,9.32 %,2.56 %,2.56 %,2.56 %,14.24 %,0.00 %,0.00 %,100.00 %,100.00 %,100.00 %
18,0.03 %,0.03 %,0.00 %,0.00 %,2.46 %,2.46 %,2.46 %,2.46 %,3.55 %,0.00 %,0.00 %,100.00 %,100.00 %,100.00 %
19,0.07 %,0.00 %,1.62 %,0.00 %,1.76 %,1.76 %,1.76 %,1.76 %,2.54 %,0.00 %,0.00 %,100.00 %,100.00 %,100.00 %
20,0.13 %,0.05 %,0.00 %,0.00 %,1.42 %,1.42 %,1.42 %,1.42 %,1.37 %,0.00 %,0.00 %,100.00 %,1.49 %,1.49 %
21,0.00 %,0.00 %,0.00 %,0.00 %,4.71 %,4.71 %,4.71 %,4.71 %,4.68 %,0.00 %,0.00 %,100.00 %,4.80 %,4.80 %
22,0.00 %,0.00 %,0.00 %,0.00 %,3.83 %,3.83 %,3.83 %,3.83 %,3.79 %,0.00 %,0.00 %,4.28 %,3.89 %,3.89 %


#### Memory

In [5]:
(idf
 .memory_usage(deep=True)
 .apply(lambda s: s / 1000000)
 .pipe(lambda df_: pd.concat([df_, idf.dtypes], axis = 1))
 .rename(columns={0:'memory', 1:'dtype'})
)

,memory,dtype
Index,0.000128,NaN
usu_estado_civil,31.269266,object
usu_fecha_nac,37.121401,object
ins_sexo,33.766269,object
ins_fecha,37.426809,object
pais_res,33.633448,object
cod_provincia_reside,4.316280,float64
cod_canton_reside,4.316280,float64
cod_parroquia_reside,4.316280,float64
ins_autoidentificacion,34.454555,object


### Columns' names and Index

In [6]:
# drop one column, rename some, for now the df does not have any meaningful index
(idf
 .rename(columns={'usu_estado_civil': 'est_civil',
                  'usu_fecha_nac':'fec_naci',
                  'ins_sexo':'sexo',
                  'ins_fecha': 'fec_insc', # fecha de inscripcion
                  'ins_autoidentificacion': 'etnia'})
)

,est_civil,fec_naci,sexo,fec_insc,pais_res,cod_provincia_reside,cod_canton_reside,cod_parroquia_reside,etnia,cod_final,periodo,escolar_no_escolar,internet_domicilio,computadora_domicilio
0,S,10/5/1998 0:00:00,MUJER,25/4/2019 18:30:37,ECUADOR,10.0,1006.0,100650.0,MESTIZO,1.416621e+09,18,NaN,NaN,NaN
1,S,09/04/2002 0:00,HOMBRE,15/01/2021 1:03,ECUADOR,9.0,922.0,92250.0,Montubio/a,2.397811e+09,21,NaN,SI,NO
2,S,20/09/2002 0:00,MUJER,17/01/2021 1:35,ECUADOR,4.0,401.0,40102.0,Mestizo/a,2.548550e+09,21,NaN,SI,NO
3,S,12/06/1985,MUJER,07/12/2019,ECUADOR,9.0,901.0,90114.0,MULATO,2.362461e+09,19,NaN,NaN,NaN
4,S,06/05/2002,MUJER,05/12/2019,ECUADOR,9.0,919.0,91953.0,MONTUBIO,2.428021e+09,19,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539530,S,24/11/2002 0:00,HOMBRE,15/01/2021 15:37,ECUADOR,17.0,1701.0,170121.0,Mestizo/a,2.503882e+09,21,NaN,SI,NO
539531,S,13/02/2000,HOMBRE,02/12/2019,ECUADOR,23.0,2302.0,230250.0,MESTIZO,1.762732e+09,19,NaN,NaN,NaN
539532,S,32756.0,HOMBRE,44047.943,NaN,NaN,NaN,NaN,NaN,4.383500e+09,20,NaN,NaN,NaN
539533,S,22/12/1987 0:00:00,MUJER,7/7/2021 21:26:15,ECUADOR,9.0,901.0,90114.0,Mestizo/a,8.594771e+09,22,No escolar,SI,SI


### Dates

In [7]:
(idf
 .rename(columns={'usu_estado_civil': 'est_civil',
                  'usu_fecha_nac':'fec_naci',
                  'ins_sexo':'sexo',
                  'ins_fecha': 'fec_insc', # fecha de inscripcion
                  'ins_autoidentificacion': 'etnia'})
 .assign(fec_naci=lambda df_: (df_
                               .fec_naci
                               .mask(df_.periodo!=20,
                                     pd.to_datetime(df_.fec_naci, 
                                                    format = '%d/%m/%Y', 
                                                    errors='coerce', 
                                                    exact = False))
                               .mask(df_.periodo==20,
                                     pd.to_datetime(df_.fec_naci.where(df_.periodo==20, 45042).astype(float), # 45042 days have transpired since 1900-01-01
                                                    unit='d', 
                                                    origin='1900-01-01'))
                              ),
         fec_insc=lambda df_: (df_
                               .fec_insc
                               .mask(df_.periodo!=20,
                                     pd.to_datetime(df_
                                                    .fec_insc, 
                                                    format='%d/%m/%Y', 
                                                    errors='coerce', 
                                                    exact=False))
                               .mask(df_.periodo==20,
                                     pd.to_datetime(df_.fec_insc.where(df_.periodo==20, 45042).astype(float), 
                                                    unit='d', 
                                                    origin='1900-01-01'))
                              ))
)

,est_civil,fec_naci,sexo,fec_insc,pais_res,cod_provincia_reside,cod_canton_reside,cod_parroquia_reside,etnia,cod_final,periodo,escolar_no_escolar,internet_domicilio,computadora_domicilio
0,S,1998-05-10 00:00:00,MUJER,2019-04-25 00:00:00,ECUADOR,10.0,1006.0,100650.0,MESTIZO,1.416621e+09,18,NaN,NaN,NaN
1,S,2002-04-09 00:00:00,HOMBRE,2021-01-15 00:00:00,ECUADOR,9.0,922.0,92250.0,Montubio/a,2.397811e+09,21,NaN,SI,NO
2,S,2002-09-20 00:00:00,MUJER,2021-01-17 00:00:00,ECUADOR,4.0,401.0,40102.0,Mestizo/a,2.548550e+09,21,NaN,SI,NO
3,S,1985-06-12 00:00:00,MUJER,2019-12-07 00:00:00,ECUADOR,9.0,901.0,90114.0,MULATO,2.362461e+09,19,NaN,NaN,NaN
4,S,2002-05-06 00:00:00,MUJER,2019-12-05 00:00:00,ECUADOR,9.0,919.0,91953.0,MONTUBIO,2.428021e+09,19,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539530,S,2002-11-24 00:00:00,HOMBRE,2021-01-15 00:00:00,ECUADOR,17.0,1701.0,170121.0,Mestizo/a,2.503882e+09,21,NaN,SI,NO
539531,S,2000-02-13 00:00:00,HOMBRE,2019-12-02 00:00:00,ECUADOR,23.0,2302.0,230250.0,MESTIZO,1.762732e+09,19,NaN,NaN,NaN
539532,S,1989-09-07 00:00:00,HOMBRE,2020-08-06 22:37:55.200000,NaN,NaN,NaN,NaN,NaN,4.383500e+09,20,NaN,NaN,NaN
539533,S,1987-12-22 00:00:00,MUJER,2021-07-07 00:00:00,ECUADOR,9.0,901.0,90114.0,Mestizo/a,8.594771e+09,22,No escolar,SI,SI


### Numeric Types

In [8]:
_df = (idf
 .rename(columns={'usu_estado_civil': 'est_civil',
                  'usu_fecha_nac':'fec_naci',
                  'ins_sexo':'sexo',
                  'ins_fecha': 'fec_insc', # fecha de inscripcion
                  'ins_autoidentificacion': 'etnia'})      
 .assign(periodo=lambda df_: df_.periodo.astype('uint8'),
         **{c:lambda df_, c=c: df_[c].astype('uint32', errors='ignore') for c in ['cod_provincia_reside', 
                                                                 'cod_canton_reside', 
                                                                 'cod_parroquia_reside']},
         fec_naci=lambda df_: (df_
                               .fec_naci
                               .mask(df_.periodo!=20,
                                     pd.to_datetime(df_.fec_naci, 
                                                    format = '%d/%m/%Y', 
                                                    errors='coerce', 
                                                    exact = False))
                               .mask(df_.periodo==20,
                                     pd.to_datetime(df_.fec_naci.where(df_.periodo==20, 45042).astype(float),
                                                    unit='d', 
                                                    origin='1900-01-01'))
                               .pipe(lambda s: pd.to_datetime(s))
                              ),
         fec_insc=lambda df_: (df_
                               .fec_insc
                               .mask(df_.periodo!=20,
                                     pd.to_datetime(df_
                                                    .fec_insc, 
                                                    format='%d/%m/%Y', 
                                                    errors='coerce', 
                                                    exact=False))
                               .mask(df_.periodo==20,
                                     pd.to_datetime(df_.fec_insc.where(df_.periodo==20, 45042).astype(float), 
                                                    unit='d', 
                                                    origin='1900-01-01'))
                               .pipe(lambda s: pd.to_datetime(s))
                              ),
         edad=lambda df_: df_.fec_insc.dt.year - df_.fec_naci.dt.year,
         mes_insc=lambda df_: df_.fec_insc.dt.month)
)

In [22]:
(_df
 .select_dtypes('number')
 .describe()
)

,cod_provincia_reside,cod_canton_reside,cod_parroquia_reside,cod_final,periodo,edad,mes_insc
count,524411.000000,524411.000000,524411.000000,5.244110e+05,524411.000000,523714.000000,524411.000000
mean,11.738966,1177.367445,117761.459683,-2.220959e+08,18.950072,21.919800,6.466207
std,5.571639,556.623602,55664.390839,1.929279e+09,1.845916,5.956129,3.913565
min,1.000000,101.000000,10101.000000,-2.147484e+09,17.000000,0.000000,1.000000
25%,9.000000,901.000000,90110.000000,-2.147484e+09,17.000000,18.000000,3.000000
50%,10.000000,1004.000000,100455.000000,1.021412e+09,19.000000,20.000000,7.000000
75%,17.000000,1701.000000,170110.000000,1.741771e+09,21.000000,23.000000,11.000000
max,90.000000,9007.000000,900751.000000,2.147482e+09,22.000000,85.000000,12.000000


### Non Numeric Types

In [11]:
with open('../../utils.json') as f:
    dd = json.load(f)

_df = (idf
 .rename(columns={'usu_estado_civil': 'est_civil',
                  'usu_fecha_nac':'fec_naci',
                  'ins_sexo':'sexo',
                  'ins_fecha': 'fec_insc', # fecha de inscripcion
                  'ins_autoidentificacion': 'etnia'})      
 .assign(periodo=lambda df_: df_.periodo.astype('uint8'),
         **{c:lambda df_, c=c: df_[c].astype('uint32', errors='ignore') for c in ['cod_provincia_reside', 
                                                                 'cod_canton_reside', 
                                                                 'cod_parroquia_reside']},
         sexo=lambda df_: df_.sexo.replace({'SIN DATO': np.nan}).str.title().astype('category'),
         pais_res=lambda df_: df_.pais_res.str.title().astype('category'),
         etnia=lambda df_: df_.etnia.str.lower().replace(regex=dd['etnias']).str.title().astype('category'),
         escolar_no_escolar=lambda df_: df_.escolar_no_escolar.astype('category'),
         **{c:lambda df_, c=c: df_[c].astype('category') for c in ['est_civil', 'internet_domicilio', 
                                                                   'computadora_domicilio']},
         fec_naci=lambda df_: (df_
                               .fec_naci
                               .mask(df_.periodo!=20,
                                     pd.to_datetime(df_.fec_naci, 
                                                    format = '%d/%m/%Y', 
                                                    errors='coerce', 
                                                    exact = False))
                               .mask(df_.periodo==20,
                                     pd.to_datetime(df_.fec_naci.where(df_.periodo==20, 45042).astype(float),
                                                    unit='d', 
                                                    origin='1900-01-01'))
                               .pipe(lambda s: pd.to_datetime(s))
                              ),
         fec_insc=lambda df_: (df_
                               .fec_insc
                               .mask(df_.periodo!=20,
                                     pd.to_datetime(df_
                                                    .fec_insc, 
                                                    format='%d/%m/%Y', 
                                                    errors='coerce', 
                                                    exact=False))
                               .mask(df_.periodo==20,
                                     pd.to_datetime(df_.fec_insc.where(df_.periodo==20, 45042).astype(float), 
                                                    unit='d', 
                                                    origin='1900-01-01'))
                               .pipe(lambda s: pd.to_datetime(s))
                              ),
         edad=lambda df_: (df_.fec_insc.dt.year - df_.fec_naci.dt.year).astype('uint16', errors='ignore'),
         mes_insc=lambda df_: (df_.fec_insc.dt.month.astype('category')
                                   .cat.add_categories([2, 6, 9, 10])
                                   .cat.reorder_categories(list(range(1, 13)), 
                                                           ordered=True)))
)

In [31]:
(_df
 .select_dtypes('category')
 .describe()
)

,est_civil,sexo,pais_res,etnia,escolar_no_escolar,internet_domicilio,computadora_domicilio
count,523555,486565,511491,500577,66749,196760,196760
unique,5,2,4,7,2,2,2
top,S,Mujer,Ecuador,Mestizo,No escolar,SI,SI
freq,478930,261299,511480,415913,53211,179668,161101


### Resulting Changes

#### Missing Values

In [12]:
(_df
 .isna()
 .groupby(_df.periodo)
 .mean()
 .style
 .format(lambda fl: f'{fl * 100:.2f} %')
)

,est_civil,fec_naci,sexo,fec_insc,pais_res,cod_provincia_reside,cod_canton_reside,cod_parroquia_reside,etnia,cod_final,periodo,escolar_no_escolar,internet_domicilio,computadora_domicilio,edad,mes_insc
periodo,,,,,,,,,,,,,,,,
17,0.40 %,0.37 %,20.86 %,0.00 %,9.32 %,2.56 %,2.56 %,2.56 %,14.24 %,0.00 %,0.00 %,100.00 %,100.00 %,100.00 %,0.37 %,0.00 %
18,0.03 %,0.03 %,1.34 %,0.00 %,2.46 %,2.46 %,2.46 %,2.46 %,3.55 %,0.00 %,0.00 %,100.00 %,100.00 %,100.00 %,0.03 %,0.00 %
19,0.07 %,0.00 %,1.62 %,0.00 %,1.76 %,1.76 %,1.76 %,1.76 %,2.54 %,0.00 %,0.00 %,100.00 %,100.00 %,100.00 %,0.00 %,0.00 %
20,0.13 %,0.05 %,0.00 %,0.00 %,1.42 %,1.42 %,1.42 %,1.42 %,1.37 %,0.00 %,0.00 %,100.00 %,1.49 %,1.49 %,0.05 %,0.00 %
21,0.00 %,0.00 %,0.00 %,0.00 %,4.71 %,4.71 %,4.71 %,4.71 %,4.68 %,0.00 %,0.00 %,100.00 %,4.80 %,4.80 %,0.00 %,0.00 %
22,0.00 %,0.00 %,0.00 %,0.00 %,3.83 %,3.83 %,3.83 %,3.83 %,3.79 %,0.00 %,0.00 %,4.28 %,3.89 %,3.89 %,0.00 %,0.00 %


#### Memory

In [13]:
(_df
 .memory_usage(deep=True)
 .apply(lambda s: s / 1000000)
 .pipe(lambda df_: pd.concat([df_, _df.dtypes], axis = 1))
 .rename(columns={0:'memory', 1:'dtype'})
)

,memory,dtype
Index,0.000128,NaN
est_civil,0.539997,category
fec_naci,4.316280,datetime64[ns]
sexo,0.539768,category
fec_insc,4.316280,datetime64[ns]
pais_res,0.540316,category
cod_provincia_reside,4.316280,float64
cod_canton_reside,4.316280,float64
cod_parroquia_reside,4.316280,float64
etnia,0.540312,category


### Tweak Inscripciones

In [32]:
with open('../utils.json') as f:
    dd = json.load(f)

def tweak_inscripciones(df):
    """
    
    Parameters:
    ----------
    `pd.DataFrame`: an instance of pd.DataFrame with the raw contents of the table 'inscripciones'
    """
    return (df
     .rename(columns={'usu_estado_civil': 'est_civil',
                      'usu_fecha_nac':'fec_naci',
                      'ins_sexo':'sexo',
                      'ins_fecha': 'fec_insc', # fecha de inscripcion
                      'ins_autoidentificacion': 'etnia'})       
     .assign(periodo=lambda df_: df_.periodo.astype('uint8'),
             **{c:lambda df_, c=c: df_[c].astype('uint32', errors='ignore') for c in ['cod_provincia_reside', 
                                                                                      'cod_canton_reside', 
                                                                                      'cod_parroquia_reside']},
             sexo=lambda df_: df_.sexo.replace({'SIN DATO': np.nan}).str.title().astype('category'),
             pais_res=lambda df_: df_.pais_res.str.title().astype('category'),
             etnia=lambda df_: df_.etnia.str.lower().replace(regex=dd['etnias']).str.title().astype('category'),
             escolar_no_escolar=lambda df_: df_.escolar_no_escolar.astype('category'),
             **{c:lambda df_, c=c: df_[c].astype('category') for c in ['est_civil', 'internet_domicilio', 
                                                                       'computadora_domicilio']},
             fec_naci=lambda df_: (df_
                                   .fec_naci
                                   .mask(df_.periodo!=20,
                                         pd.to_datetime(df_.fec_naci, 
                                                        format = '%d/%m/%Y', 
                                                        errors='coerce', 
                                                        exact = False))
                                   .mask(df_.periodo==20,
                                         pd.to_datetime(df_.fec_naci.where(df_.periodo==20, 45042).astype(float),
                                                        unit='d', 
                                                        origin='1900-01-01'))
                                   .pipe(lambda s: pd.to_datetime(s))
                                  ),
             fec_insc=lambda df_: (df_
                                   .fec_insc
                                   .mask(df_.periodo!=20,
                                         pd.to_datetime(df_
                                                        .fec_insc, 
                                                        format='%d/%m/%Y', 
                                                        errors='coerce', 
                                                        exact=False))
                                   .mask(df_.periodo==20,
                                         pd.to_datetime(df_.fec_insc.where(df_.periodo==20, 45042).astype(float), 
                                                        unit='d', 
                                                        origin='1900-01-01'))
                                   .pipe(lambda s: pd.to_datetime(s))
                                  ),
             edad=lambda df_: df_.fec_insc.dt.year - df_.fec_naci.dt.year,
             mes_insc=lambda df_: (df_.fec_insc.dt.month.astype('category')
                                   .cat.add_categories([2, 6, 9, 10])
                                   .cat.reorder_categories(list(range(1, 13)), 
                                                           ordered=True)))
    )